In [39]:
import datetime as dt
import numpy as np
import math
import matplotlib.pyplot as plt
from __future__ import print_function

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from multiprocess import Process
from multiprocess import Queue
import multiprocess as mp
import time
import random

INFO:tensorflow:Disabling eager execution
INFO:tensorflow:Disabling v2 tensorshape
INFO:tensorflow:Disabling resource variables
INFO:tensorflow:Disabling tensor equality
INFO:tensorflow:Disabling control flow v2


In [61]:
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped
class Network:
    def __init__(self):
        self.layers = []
        self.tn = Neuron(100)
    
    def add(self, layer):
        self.layers.append(layer)
    
    def full_connect(self):
        for i in range(len(self.layers)):
            for neuron in self.layers[i].neurons:
                if i + 1 < len(self.layers):
                    for r_neuron in self.layers[i + 1].neurons:
                        neuron.add_recipient(r_neuron)
                else:
                    neuron.add_recipient(self.tn)
  
    def train(self, full_inputs, full_outputs, time_per_image, worker_amount):
        results = []
        if len(full_inputs) != len(full_outputs):
            print('you have done something wrong!')
            return None
        for layer in self.layers:
            layer.set_update(True)
        for i in range(len(full_inputs)):
            self.layers[-1].set_training_outputs(full_outputs[i])
            for _ in range(time_per_image):
                self.layers[0].receive_inputs(full_inputs[i])
                for j in range(1, len(self.layers) - 1):
                    self.layers[j].fire()
                results.append(self.layers[-1].fire())
                print(results[-1], full_outputs[i])
        return results
    
    def predict(self, full_inputs, time_per_image, worker_amount):
        results = []
        for layer in self.layers:
            layer.set_update(False)
        for i in range(len(full_inputs)):
            for _ in range(time_per_image):
                self.layers[0].receive_inputs(full_inputs[i])
                for j in range(1, len(self.layers) - 1):
                    self.layers[j].fire()
                results.append(self.layers[-1].fire())
                print(results[-1], full_outputs[i])
        return results
            

        
    def receive_inputs(self, inputs):
        if len(inputs) != len(self.layers[0]):
            print('input len != len of first layer')
        self.layers[0].receive_input(inputs)

    def set_training_outputs(self, desired_outputs):
        if len(desired_outputs) != len(self.layers[-1]):
            print('input len != len of first layer')
        self.layers[0].set_training_outputs(set_training_outputs)
        
    
    def sparse_connect(self):
        pass
        

In [62]:
class Layer:
    def __init__(self, size):
        self.neurons = [Neuron(i) for i in range(size)]
    
    def fire(self):
        firing = []
        for neuron in self.neurons:
            fs = [func() for func in neuron.attempt_fire()]
            firing.append(len(fs))
            print(fs)
        return [1 if f > 0 else 0 for f in firing]
    
    def receive_inputs(self, inputs):
        for i in range(len(inputs)):
            f = [func() for func in self.neurons[i].receive_input(inputs[i])]


    def set_training_outputs(self, outputs):
        for i in range(len(outputs)):
            self.neurons[i].set_outputs(outputs[i])

    def set_update(self, b):
        for neuron in self.neurons:
            neuron.set_weight_update(b)

In [63]:
class Neuron:
    def __init__(self, num):
        # a unique identifier
        self.num = num
        # When the neuron will fire
        self.action_potential = 105.0
        # The membrane potential when it was last checked
        self.membrane_potential = 0
        # references to neurons that will increase this one's potential
        self.incomming_connections = {}
        # references to neurons that will be increased by this one
        self.outgoing_connections = []
        # resting potential
        self.resting_potential = 0
        # leak ammount
        self.leak = 0.2
        # last fire
        self.last_fire = dt.datetime.now()
        # how long the neron has to physically wait befor it can fire again.
        self.fire_rate = dt.timedelta(microseconds=500)
        # expected output
        self.expected_output = None
        self.update_weights = True
    
    def set_weight_update(self, b):
        self.update_weights = b

    def set_outputs(self, output):
        self.expected_output = output
        
    
    def add_recipient(self, neuron):
        """
        Add a connection to a new neuron.
        """
        self.outgoing_connections.append(neuron)
    
    def leak(self):
        """
        A function that will cause a constant leakage of membrane potential.
        This keeps the neuron near "equilibrium".
        """
        if self.membrane_potential < self.resting_potential:
            # If we are below the resting potential we rise to it.
            self.membrane_potential += self.leak
        else:
            # If we are above the resting potential we lower to it.
            self.membrane_potential -= self.leak
    def attempt_fire(self):
        if (dt.datetime.now() - self.last_fire) > self.fire_rate and self.membrane_potential >= self.action_potential:
            return self.fire()
        return []

    def fire(self):
        self.last_fire = dt.datetime.now()
        self.membrane_potential = -10
        return [wrapper(c.receive_input, self.num) for c in self.outgoing_connections]  
        
    
    def receive_input(self, amount):
        self.membrane_potential += amount
        if (dt.datetime.now() - self.last_fire) > self.fire_rate and self.membrane_potential >= self.action_potential:
            return self.fire()
        return []
    
    def receive_fire(self, neuron):
        """To be used with multiprocessing """
        weight = self.incomming_connections.get(neuron)
        if weight is not None:
            self.membrane_potential += weight
        else:
            # setting initial weight
            self.incomming_connections[neuron] = random.random() * 10 + 10
            self.membrane_potential += self.incomming_connections[neuron]
            
        if (dt.datetime.now() - self.last_fire) > self.fire_rate and self.membrane_potential >= self.action_potential:
            if self.expected_output != None:
                self.output_queue.put(self.num)
                if self.expected_output == 0 and self.update_weights:
                    self.incomming_connections[neuron] -= 1.5
            if (self.update_weights):
                self.incomming_connections[neuron] += 1
            return self.fire()
        if self.update_weights:
            self.incomming_connections[neuron] -= 0.1 
        return []

In [64]:
neuron0 = Neuron(0)
neuron1 = Neuron(1)
neuron0.add_recipient(neuron1)
now = dt.datetime.now()

for i in range(1000):
    # print(neuron0.membrane_potential)
    if len(neuron0.receive_input(1)) > 0:
        print(f'fire {dt.datetime.now() - now}')

fire 0:00:00.000376
fire 0:00:00.000878
fire 0:00:00.001379
fire 0:00:00.001882


In [65]:
data = [[random.random() for __ in range(784)] for _ in range(10)]
labels = [[1 if random.random() > 0.5 else 0 for __ in range(2)] for _ in range(10)]


In [66]:
model = Network()
model.add(Layer(784))
model.add(Layer(2))
model.full_connect()

In [67]:
print(model.train(data, labels, 500, 6))

[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]


[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0

[]
[[]]
[0, 1] [0, 1]
[[]]
[[]]
[1, 1] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[[]]
[]
[1, 0] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[]
[0, 0] [0, 1]
[[]]
[[]]
[1, 1] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[]
[0, 0] [0, 1]
[[]]
[[]]
[1, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[[]]
[]
[1, 0] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[]
[0, 0] [0, 1]
[[]]
[[]]
[1, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[[]]
[[]]
[1, 1] [0, 1]
[[]]
[]
[1, 0] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[]
[0, 0] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 1] [0, 1]
[]
[[]]
[0, 

[[]]
[[]]
[1, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1,

[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1,

...

[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1

[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]

[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[[]]
[]
[1, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[]
[]
[0, 0] [1, 0]
[]
[]
[0, 0] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[]
[[]]
[0, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]
[[]]
[[]]
[1, 1] [1, 0]


[]
[[]]
[0, 1] [1, 1]
[[]]
[]
[1, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[]
[1, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[[]]
[]
[1, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [

[]
[[]]
[0, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[[]]
[]
[1, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[[]]
[]
[1, 0] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[[]]
[0, 1] [1, 1]
[[]]
[[]]
[1, 1] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[]
[0, 0] [1, 1]
[]
[[]]
